In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

%matplotlib inline
pd.set_option('display.max_rows', 500)


In [2]:
# pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
# pd_JH_data=pd_JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()
# pd_JH_data.head()

In [3]:

pd_JH_data_country=pd.read_csv('../data/processed/COVID_relational_database_by_country_confirmed.csv',sep=';',parse_dates=[0])
pd_JH_data_country=pd_JH_data_country.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data_country.head()

,date,country,confirmed
0,2020-01-22,Afghanistan,0
1,2020-01-22,Namibia,0
2,2020-01-22,Nepal,0
3,2020-01-22,Netherlands,0
4,2020-01-22,New Zealand,0


In [4]:
# len(set(pd_JH_data["country"]))== len(set(pd_JH_data_country["country"]))

In [5]:
test_data=pd_JH_data_country[((pd_JH_data_country['country']=='Nepal')|
                      (pd_JH_data_country['country']=='Germany'))&
                     (pd_JH_data_country['date']>'2020-03-20')
                   ]


In [6]:
test_data.head()


,date,country,confirmed
11094,2020-03-21,Nepal,1
11272,2020-03-21,Germany,22213
11282,2020-03-22,Nepal,1
11460,2020-03-22,Germany,24873
11471,2020-03-23,Nepal,2


In [7]:
test_data.groupby(['country']).agg(np.max)

,date,confirmed
country,,
Germany,2020-09-14,263222
Nepal,2020-09-14,55329


In [8]:
import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)
from scipy import signal

def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope




In [9]:
def rolling_reg(df_input,col='confirmed'):
    ''' input has to be a data frame'''
    ''' return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result
    

In [10]:
test_data[['country','confirmed']].groupby(['country']).apply(rolling_reg,'confirmed')

country       
Germany  11272            NaN
         11460            NaN
         11654       7.417994
         11835       7.142035
         12024       8.012983
         12215       6.954407
         12406       6.501919
         12594       7.390371
         12777      10.136671
         12963      13.541893
         13152      13.781393
         13342      13.140681
         13533      12.037271
         13719      12.735506
         13910      16.052694
         14094      21.372453
         14280      27.427355
         14471      27.511936
         14656      21.792179
         14848      21.495848
         15035      26.565108
         15223      36.198404
         15408      43.982990
         15600      49.423444
         15784      74.043747
         15976      56.424411
         16163      42.468318
         16352      41.525988
         16539      49.898063
         16727      75.684007
         16912      78.000000
         17104      94.526338
         17292      82.98

## applying to large dataset

In [11]:
pd_DR_result_country=pd_JH_data_country[['country','confirmed']].groupby(['country']).apply(rolling_reg,'confirmed').reset_index()

In [12]:
pd_DR_result_country.head()

,country,level_1,confirmed
0,Afghanistan,0,NaN
1,Afghanistan,329,NaN
2,Afghanistan,496,NaN
3,Afghanistan,683,NaN
4,Afghanistan,871,NaN


In [13]:
pd_DR_result_country=pd_DR_result_country.rename(columns={'confirmed':'confirmed_DR',
                             'level_1':'index'})
pd_DR_result_country.tail()

,country,index,confirmed_DR
44551,Zimbabwe,43681,228.410256
44552,Zimbabwe,43849,298.146667
44553,Zimbabwe,44057,272.000000
44554,Zimbabwe,44204,319.333333
44555,Zimbabwe,44555,654.057971


In [14]:
pd_JH_data_country.head()

,date,country,confirmed
0,2020-01-22,Afghanistan,0
1,2020-01-22,Namibia,0
2,2020-01-22,Nepal,0
3,2020-01-22,Netherlands,0
4,2020-01-22,New Zealand,0


In [15]:
pd_JH_data_country=pd_JH_data_country.reset_index()
pd_JH_data_country.head()

,index,date,country,confirmed
0,0,2020-01-22,Afghanistan,0
1,1,2020-01-22,Namibia,0
2,2,2020-01-22,Nepal,0
3,3,2020-01-22,Netherlands,0
4,4,2020-01-22,New Zealand,0


In [16]:
pd_result_large_country=pd.merge(pd_JH_data_country,pd_DR_result_country[['index','confirmed_DR']],on=['index'],how='left')
pd_result_large_country.tail()

,index,date,country,confirmed,confirmed_DR
44551,44551,2020-09-14,Gambia,3405,234.160920
44552,44552,2020-09-14,Georgia,2392,14.077813
44553,44553,2020-09-14,Germany,263222,217.817325
44554,44554,2020-09-14,Estonia,2698,124.480620
44555,44555,2020-09-14,Zimbabwe,7531,654.057971


## filter data using groupby and apply


In [17]:
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result

In [18]:
pd_filtered_result_country=pd_JH_data_country[['country','confirmed']].groupby(['country']).apply(savgol_filter).reset_index()

In [19]:
pd_result_large_country=pd.merge(pd_result_large_country,pd_filtered_result_country[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_large_country.tail()

,index,date,country,confirmed,confirmed_DR,confirmed_filtered
44551,44551,2020-09-14,Gambia,3405,234.160920,3414.2
44552,44552,2020-09-14,Georgia,2392,14.077813,2375.0
44553,44553,2020-09-14,Germany,263222,217.817325,263161.6
44554,44554,2020-09-14,Estonia,2698,124.480620,2700.2
44555,44555,2020-09-14,Zimbabwe,7531,654.057971,7540.0


### Filtered doubling rate

In [20]:

pd_filtered_doubling_country=pd_result_large_country[['country','confirmed_filtered']].groupby(['country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling_country=pd_filtered_doubling_country.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_1':'index'})

pd_filtered_doubling_country.tail()

,country,index,confirmed_filtered_DR
44551,Zimbabwe,43681,86.626761
44552,Zimbabwe,43849,213.982759
44553,Zimbabwe,44057,311.525000
44554,Zimbabwe,44204,368.519247
44555,Zimbabwe,44555,370.428571


In [21]:
pd_result_large_country=pd.merge(pd_result_large_country,pd_filtered_doubling_country[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_large_country.tail()

,index,date,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
44551,44551,2020-09-14,Gambia,3405,234.160920,3414.2,175.901554
44552,44552,2020-09-14,Georgia,2392,14.077813,2375.0,15.562064
44553,44553,2020-09-14,Germany,263222,217.817325,263161.6,215.629569
44554,44554,2020-09-14,Estonia,2698,124.480620,2700.2,111.508333
44555,44555,2020-09-14,Zimbabwe,7531,654.057971,7540.0,370.428571


In [22]:
mask=pd_result_large_country['confirmed']>100
pd_result_large_country['confirmed_filtered_DR']=pd_result_large_country['confirmed_filtered_DR'].where(mask, other=np.NaN) 


In [23]:
pd_result_large_country[pd_result_large_country['country']=='Germany'].tail()

,index,date,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
43800,43800,2020-09-10,Germany,258149,160.722431,258018.2,168.789051
43984,43984,2020-09-11,Germany,259735,156.332930,259374.2,184.661656
44176,44176,2020-09-12,Germany,260817,194.577961,260732.0,191.152480
44360,44360,2020-09-13,Germany,261737,260.502498,261946.8,202.662158
44553,44553,2020-09-14,Germany,263222,217.817325,263161.6,215.629569


In [24]:
pd_result_large_country.to_csv('../data/processed/COVID_final_set_country.csv',sep=';',index=False)